In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [89.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-12 04:01:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.51MB/s    in 0.2s    

2022-08-12 04:01:31 (4.51 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in dataset from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

appliances_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
appliances_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [5]:
print(f'Total appliances product reviews: {appliances_df.count()}')

Total appliances product reviews: 96901


In [6]:
from pyspark.sql.functions import *

# Make outdoors review id dataframe and convert review date variable to date
appliances_review_ids = appliances_df.select(col("review_id"),col("customer_id"),col("product_id"),
                               col("product_parent"),to_date(col("review_date"),"yyyy-mm-dd").alias('review_date'))

In [7]:
# Verify datatypes
appliances_review_ids.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [8]:
# View table
appliances_review_ids.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R203HPW78Z7N4K|   16199106|B0067WNSZY|     633038551| 2015-01-31|
|R2EAIGVLEALSP3|   16374060|B002QSXK60|     811766671| 2015-01-31|
|R1K1CD73HHLILA|   15322085|B00EC452R6|     345562728| 2015-01-31|
|R2KZBMOFRMYOPO|   32004835|B00MVVIF2G|     563052763| 2015-01-31|
| R6BIZOZY6UD01|   25414497|B00IY7BNUW|     874236579| 2015-01-31|
|R1MCXZFNF8E7Y0|   36311751|B0033X29CI|     294467812| 2015-01-31|
|R3EMB3E3ODR6BW|   30920961|B005R597HA|     183784715| 2015-01-31|
| RJTONVTTOPJ5S|   52491265|B00MO6V8Y0|     960251524| 2015-01-31|
|R21U5QZ2CQECUM|   48166169|B00HT39QDI|     992475314| 2015-01-31|
| RL2BBC51H89DH|   50394924|B00LESFZ52|       1641606| 2015-01-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [9]:
# Make appliances products table
appliances_products = appliances_df[['product_id','product_title']]

In [10]:
# Check number of rows
print(f'Number of rows before duplicates removed: {appliances_products.count()}')

# Drop duplicate rows
appliances_products = appliances_products.drop_duplicates()

# Check number of rows after duplicates removed 
print(f'Number of rows after duplicates removed: {appliances_products.count()}')

Number of rows before duplicates removed: 96901
Number of rows after duplicates removed: 11694


In [11]:
# Check if there are more than one listing of a certain product id
appliances_products\
        .groupby('product_id')\
        .agg(count('product_id').alias('product_count'))\
        .sort(desc('product_count'))\
        .show()

+----------+-------------+
|product_id|product_count|
+----------+-------------+
|B0011YJE7Y|            1|
|B00C4TJIQM|            1|
|B000ZINOYG|            1|
|B004COFFGO|            1|
|B005BHT0KI|            1|
|B00VIVR7O2|            1|
|B00GWRWIOG|            1|
|B00COPSKHY|            1|
|B00291CQCI|            1|
|B00EP02WAC|            1|
|B00IGBTZDI|            1|
|B00KTBQNTC|            1|
|B001DHRD2M|            1|
|B00AQ99BTQ|            1|
|B00CVT7G9Q|            1|
|B001F8GIX4|            1|
|B00E05AW3G|            1|
|B00727TRQM|            1|
|B0049UXKJA|            1|
|B00LXITA86|            1|
+----------+-------------+
only showing top 20 rows



In [12]:
# Create appliances customers table and count number of times each customer has left a review
appliances_customers = appliances_df.groupby("customer_id").count()

In [13]:
# View table
appliances_customers.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   13326061|    1|
|   52512151|    1|
|    8968933|    1|
|   39416583|    1|
|   21737233|    1|
|   44119972|    1|
|   13947800|    1|
|   21482558|    1|
|    2802853|    1|
|   18518845|    1|
|     122484|    1|
|   14035069|    1|
|   20004191|    1|
|   16699322|    1|
|   24768141|    1|
|   24424556|    1|
|   44293588|    1|
|   13188682|    1|
|   23298840|    1|
|   50731398|    1|
+-----------+-----+
only showing top 20 rows



In [14]:
# Rename count column
appliances_customers = appliances_customers.withColumnRenamed('count','customer_count')
appliances_customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13326061|             1|
|   52512151|             1|
|    8968933|             1|
|   39416583|             1|
|   21737233|             1|
|   44119972|             1|
|   13947800|             1|
|   21482558|             1|
|    2802853|             1|
|   18518845|             1|
|     122484|             1|
|   14035069|             1|
|   20004191|             1|
|   16699322|             1|
|   24768141|             1|
|   24424556|             1|
|   44293588|             1|
|   13188682|             1|
|   23298840|             1|
|   50731398|             1|
+-----------+--------------+
only showing top 20 rows



In [15]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://reviewsdb.csumjblrdtra.us-west-2.rds.amazonaws.com:5432/reviews_db"
config = {"user":"<user>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [16]:
# Write appliances_review_ids to table
appliances_review_ids.write.jdbc(url=jdbc_url, table='appliance_review_id_table', mode=mode, properties=config)

In [17]:
# Write appliances_products to table
appliances_products.write.jdbc(url=jdbc_url, table='appliance_products', mode=mode, properties=config)

In [18]:
# Write appliances_customers to table
appliances_customers.write.jdbc(url=jdbc_url, table='appliance_customers', mode=mode, properties=config)